Task_1

In [409]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import (
    mean_absolute_error,
    mean_squared_error,
    r2_score,
    mean_absolute_percentage_error,
)

ratings_df = pd.read_csv(
    "u.data.csv",
    sep="\t",
    header=None,
    names=["user_id", "item_id", "rating", "timestamp"],
)

movies_df = pd.read_csv(
    "u.item.csv",
    sep="|",
    header=None,
    names=[
        "movie_id",
        "movie_title",
        "release_date",
        "video_release_date",
        "IMDb_URL",
        "unknown",
        "Action",
        "Adventure",
        "Animation",
        "Children's",
        "Comedy",
        "Crime",
        "Documentary",
        "Drama",
        "Fantasy",
        "Film_Noir",
        "Horror",
        "Musical",
        "Mystery",
        "Romance",
        "Sci_fi",
        "Thriller",
        "War",
        "Western",
    ],
    encoding="latin1",
)

Task_2

In [410]:
max_rating_user_id = ratings_df.groupby("user_id")["rating"].count().idxmax()
print(max_rating_user_id)

405


Task_3

In [411]:
ratings_df["ttl_ratings_count"] = ratings_df.groupby("item_id")["rating"].transform(
    "count"
)
ratings_df["ttl_ratings_sum"] = ratings_df.groupby("item_id")["rating"].transform("sum")

ratings_df = ratings_df[ratings_df["user_id"] == max_rating_user_id]
ratings_df = ratings_df[["item_id", "rating", "ttl_ratings_count", "ttl_ratings_sum"]]
print(ratings_df.head())

       item_id  rating  ttl_ratings_count  ttl_ratings_sum
12276       56       4                394             1600
12383      592       1                  9               30
12430     1582       1                  1                1
12449      171       1                 65              252
12460      580       1                 32              108


Task_4

In [412]:
ratings_df = ratings_df.merge(
    movies_df, how="inner", left_on="item_id", right_on="movie_id"
)
print(ratings_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 737 entries, 0 to 736
Data columns (total 28 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   item_id             737 non-null    int64  
 1   rating              737 non-null    int64  
 2   ttl_ratings_count   737 non-null    int64  
 3   ttl_ratings_sum     737 non-null    int64  
 4   movie_id            737 non-null    int64  
 5   movie_title         737 non-null    object 
 6   release_date        737 non-null    object 
 7   video_release_date  0 non-null      float64
 8   IMDb_URL            736 non-null    object 
 9   unknown             737 non-null    int64  
 10  Action              737 non-null    int64  
 11  Adventure           737 non-null    int64  
 12  Animation           737 non-null    int64  
 13  Children's          737 non-null    int64  
 14  Comedy              737 non-null    int64  
 15  Crime               737 non-null    int64  
 16  Document

Task_5

In [413]:
ratings_df["release_date"] = pd.to_datetime(ratings_df["release_date"])
ratings_df["release_year"] = ratings_df["release_date"].dt.year

features = [
    "release_year",
    "ttl_ratings_count",
    "ttl_ratings_sum",
] + ratings_df.columns[9:].tolist()
X = ratings_df[features]
y = ratings_df["rating"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)
print(f"X_train: {X_train.shape}, y_train: {y_train.shape}")
print(f"X_test: {X_test.shape}, y_test: {y_test.shape}")

X_train: (589, 23), y_train: (589,)
X_test: (148, 23), y_test: (148,)


Task_6

In [414]:
model = LinearRegression()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

Task_7

In [415]:
print(f"MAE: {round(mean_absolute_error(y_test, y_pred),2)}")
print(f"MSE: {round(mean_squared_error(y_test, y_pred), 2)}")
print(f"RMSE: {round(np.sqrt(mean_squared_error(y_test, y_pred)),2)}")
print(f"R²: {round(r2_score(y_test, y_pred),2)}")
print(f"MAPE: {round(mean_absolute_percentage_error(y_test, y_pred) * 100, 2)}")

MAE: 0.95
MSE: 1.7
RMSE: 1.3
R²: 0.16
MAPE: 55.46


Task_8

In [416]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import avg, col, count

spark = SparkSession.builder.master("local[*]").getOrCreate()

ratings_df = spark.read.csv(
    "u.data.csv", sep="\t", header=False, inferSchema=True
).toDF("user_id", "item_id", "rating", "timestamp")


movies_df = spark.read.csv(
    "u.item.csv", sep="|", header=False, inferSchema=True, encoding="ISO-8859-1"
).toDF(
    "movie_id",
    "movie_title",
    "release_date",
    "video_release_date",
    "IMDb_URL",
    "unknown",
    "Action",
    "Adventure",
    "Animation",
    "Children's",
    "Comedy",
    "Crime",
    "Documentary",
    "Drama",
    "Fantasy",
    "Film_Noir",
    "Horror",
    "Musical",
    "Mystery",
    "Romance",
    "Sci_fi",
    "Thriller",
    "War",
    "Western",
)

print(ratings_df.dtypes)
print(movies_df.dtypes)

[('user_id', 'int'), ('item_id', 'int'), ('rating', 'int'), ('timestamp', 'int')]
[('movie_id', 'int'), ('movie_title', 'string'), ('release_date', 'string'), ('video_release_date', 'string'), ('IMDb_URL', 'string'), ('unknown', 'int'), ('Action', 'int'), ('Adventure', 'int'), ('Animation', 'int'), ("Children's", 'int'), ('Comedy', 'int'), ('Crime', 'int'), ('Documentary', 'int'), ('Drama', 'int'), ('Fantasy', 'int'), ('Film_Noir', 'int'), ('Horror', 'int'), ('Musical', 'int'), ('Mystery', 'int'), ('Romance', 'int'), ('Sci_fi', 'int'), ('Thriller', 'int'), ('War', 'int'), ('Western', 'int')]


Task_9

In [417]:
joined_df = ratings_df.join(
    movies_df,
    ratings_df["item_id"] == movies_df["movie_id"],
)

avg_ratings = (
    joined_df.groupBy("movie_id")
    .agg(avg("rating").alias("avg_rating"))
    .orderBy("avg_rating", ascending=False)
)
avg_ratings.show(5)

+--------+----------+
|movie_id|avg_rating|
+--------+----------+
|    1599|       5.0|
|    1467|       5.0|
|    1500|       5.0|
|    1653|       5.0|
|    1122|       5.0|
+--------+----------+
only showing top 5 rows



Task_10

In [418]:
genres = joined_df.columns[9:]

for genre in genres:
    avg_rating = (
        joined_df.filter(col(genre) == 1)
        .agg(avg("rating").alias("avg_rating"))
        .first()["avg_rating"]
    )
    print(f"{genre}: {round(avg_rating, 2)}")

unknown: 3.2
Action: 3.48
Adventure: 3.5
Animation: 3.58
Children's: 3.35
Comedy: 3.39
Crime: 3.63
Documentary: 3.67
Drama: 3.69
Fantasy: 3.22
Film_Noir: 3.92
Horror: 3.29
Musical: 3.52
Mystery: 3.64
Romance: 3.62
Sci_fi: 3.56
Thriller: 3.51
War: 3.82
Western: 3.61


Task_11

In [419]:
movies_popularity = joined_df.groupBy("movie_id", "movie_title").agg(
    count("rating").alias("rating_count"), avg("rating").alias("avg_rating")
)

top5_popular = movies_popularity.orderBy(col("rating_count").desc()).limit(5).show()
top5_unpopular = movies_popularity.orderBy(col("rating_count").asc()).limit(5)

+--------+--------------------+------------+------------------+
|movie_id|         movie_title|rating_count|        avg_rating|
+--------+--------------------+------------+------------------+
|      50|    Star Wars (1977)|         583|4.3584905660377355|
|     258|      Contact (1997)|         509|3.8035363457760316|
|     100|        Fargo (1996)|         508| 4.155511811023622|
|     181|Return of the Jed...|         507| 4.007889546351085|
|     294|    Liar Liar (1997)|         485| 3.156701030927835|
+--------+--------------------+------------+------------------+

